> *Premature optimization is the root of all evil.*
> <div align="right"> Donald Knuth </div>

In the analysis of freely-diffusing single-molecule experiments, 
burst search algorithm is the central step that allows identifying 
and quantify single molecule fluorescence. In this post, I use the 
sliding-window burst search algorithm 
([Fries 1998](http://dx.doi.org/10.1021/jp980965t)) as a real-world example to 
illustrate some typical optimization techniques commonly used in python, 
namely vectorization (numpy) and compilation to machine code (cython, numba).
I will use a simplified burst search that saves only start and stop times
of each burst. For a complete implementation see the 
[FRETBursts](http://tritemio.github.io/FRETBursts/) burst analysis 
software ([code here](https://github.com/tritemio/FRETBursts/tree/master/fretbursts/burstsearch)).

As of why I choose python, you can read the famous Jake Vanderplas' post: 
[Why Python is the Last Language You'll Have To Learn](https://jakevdp.github.io/blog/2012/09/20/why-python-is-the-last/). 
Or, as John Cook synthetically put it:

> I’d rather do mathematics in a general programming language than do general programming in a mathematical language.

(from John's recent post 
[Scientific computing in Python](http://www.johndcook.com/blog/2015/07/16/scientific-computing-in-python/)).

# Burst search

Briefly, a burst search algorithm tries to identifies bursts in a long stream 
of events. In single-molecule fluorescence experiments, this stream is
represented by photon arrival times (timestamps) with a resolution of a few 
nanoseconds.

A common algorithm, introduced by the Seidel group
([Fries 1998](http://dx.doi.org/10.1021/jp980965t)), consists in using a 
sliding windows of duration $T$ and identifying bursts when at least $m$ photons 
lie in this window.
The final step of selecting bursts by size (counts, or number of photons)
is computationally inexpensive and it will be ignored in this post.

Numerically, we need to "slide" the window in discrete steps, and since 
the photon arrival times are stochastic, it makes sense to place
the windows start in correspondence with each timestamp $t_i$ and check 
if there are at least $m$ photons between $t_i$ and $t_i + T$.

But how can we "check if there are at least $m$ photons between 
$t_i$ and $t_i + T$"? In principle, we can count how many photons 
lie in the current time windows and start the burst when this number 
is $\ge m$. 
This would require, for each timestamp $t_i$, looping through a variable
number of timestamps ($n$), until $t_{i+n} - t_i >=T$ then check if 
$n >= m$. In total, we would loop through the full `timestamps` array many times.

Equivalently, we can loop over consecutive m-tuple of photons, and 
test if the time difference between the last and the first timestamp 
is $\le T$. This latter approach is much easier to implement and more
efficient as it requires looping through the `timestamps` exactly once.

For the sake of this post, we assume that each burst is characterized 
by only a start and stop time. Finally, since the number of bursts 
is not known in advance, we'll use a list to collect the bursts
found during the search.

# Prepare the data

To test different burst search implementation we can use a single-molecule FRET dataset [available on figshare](http://figshare.com/articles/Example_smFRET_data_files_in_Photon_HDF5_format/1456362). The file are in [Photon-HDF5](http://photon-hdf5.org), so we can load its content with a HDF5 library, such as [pytables](http://www.pytables.org/).

For this post we only need to load the `timestamps` array, which is here converted in seconds:

In [1]:
import tables
import numpy as np

In [2]:
filename = "data/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5"

In [3]:
with tables.open_file(filename) as h5file:
    timestamps = h5file.root.photon_data.timestamps.read()
    timestamps = timestamps*h5file.root.photon_data.timestamps_specs.timestamps_unit.read()

In [4]:
timestamps

array([  1.83558750e-03,   2.35056250e-03,   3.67655000e-03, ...,
         5.99998296e+02,   5.99998472e+02,   5.99999442e+02])

In [5]:
timestamps.size

2683962

# Implementation 1: for-loop

The previously described algorithm can be expressed quite naturally with a for-loop:

In [6]:
def burst_search(times, m, T):
    in_burst = False
    bursts = []
    for i in range(times.size - m - 1):
        if times[i + m - 1] - times[i] <= T:
            if not in_burst:
                in_burst = True
                start = times[i]
        elif in_burst:
            in_burst = False
            bursts.append((start, times[i+m-1]))
    return bursts

The code is straightforward to read:

1. the $i$ variable loops over the timestamps index
2. if the $m$ consecutive photons starting at $t_i$ are within a window $\le T$
  1. if a burst is not already started, start the burst and save the start time
3. Otherwise, if we are inside a burst, stop the burst and save the stop time

Let's run it:

In [7]:
bursts_py = burst_search(timestamps, 10, 100e-6)

In [8]:
len(bursts_py)

18529

In [9]:
%timeit burst_search(timestamps, 10, 100e-6)

1 loops, best of 3: 935 ms per loop


So, we found 18529 bursts and the execution took around a second. 
It seems we got the results "fast enough". However, 
when dealing with larger files (longer measurement, multi-spot, etc...) 
or when we need to interactively explore the effect of burst serach 
parameters on burst populations
we need a faster burst search.

Next sections show various optimization approches.

# Implementation 2: for-loop compiled

First step of optimization is always identifying the 
[bottlenecks](https://en.wikipedia.org/wiki/Bottleneck_(software). 
In this case, the looping over >2 millions of elements and 
computing `times[i+m] - times[i] <= T` at each cycle is where
the function spends the most time. The list-appending
adds a negligible time since is performed once per burst
(not at each cycle).

We can confirm this analysis by running `line_profiler`
(install it through PIP or conda), which gives a line-by-line
measure of the execution time.

For a more detailed overview of different profiling techniques
see the excellent Cyrille Rossant's post 
[Profiling and optimizing Python code](http://cyrille.rossant.net/profiling-and-optimizing-python-code/).

In [10]:
%load_ext line_profiler

In [11]:
%lprun -f burst_search burst_search(timestamps, 10, 100e-6)

```
Timer unit: 1e-06 s

Total time: 4.50356 s
File: <ipython-input-6-e29550a3433a>
Function: burst_search at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def burst_search(times, m, T):
     2         1            1      1.0      0.0      in_burst = False
     3         1            1      1.0      0.0      bursts = []
     4   2683952      1166986      0.4     25.9      for i in range(times.size - m - 1):
     5   2683951      2176824      0.8     48.3          if times[i + m - 1] - times[i] <= T:
     6    232747       101596      0.4      2.3              if not in_burst:
     7     18529         7970      0.4      0.2                  in_burst = True
     8     18529        10306      0.6      0.2                  start = times[i]
     9   2451204      1017383      0.4     22.6          elif in_burst:
    10     18529         8031      0.4      0.2              in_burst = False
    11     18529        14458      0.8      0.3              bursts.append((start, times[i+m-1]))
    12         1            0      0.0      0.0      return bursts
```    
    
The profiler comfirms that >95% of the time is spent
looping and branching, (which includes computing 
`times[i + m - 1] - times[i] <= T`).
It was was surprising, for me, to find that the simple 
branching (see line 9) account for a significant fraction of 
the total time.

The `burst_search()` function has a "hot" for-loop containing 
an if-else branching that depends on array elements computations. 
This is an example where removing the loop through vectorization is 
not trivial. 
Therefore the "cure" is compiling the function 
to machine code. This can be achieved by using cython.

Cython extends the python syntax, and allows to 
statically translate python to C (that, eventually, needs to be compiled to 
machine code).  
To gain speed, we specify the types for variables used
inside the loop, loosing indeed some dynamic-typing. In this case, we know 
that `timestamps` is an float64 array.

The cython version of the previous algorithm becomes:

In [12]:
%load_ext Cython

In [13]:
%%cython
cimport numpy as np

def burst_search_cy(np.float64_t[:] times, np.int16_t m, np.float64_t T):
    cdef int i
    cdef np.float64_t start
    cdef np.uint8_t in_bursts
    
    in_burst = 0
    bursts = []
    for i in range(times.size - m - 1):
        if times[i + m - 1] - times[i] <= T:
            if not in_burst:
                in_burst = 1
                start = times[i]
        elif in_burst:
            in_burst = 0
            bursts.append((start, times[i+m-1]))
    return bursts

In the Jupyter notebook, we use the `%%cython` 
[magic command](https://ipython.org/ipython-doc/dev/interactive/tutorial.html) 
(included in the `cython` package) to easily compile the function.
Outside the notebook, we can setup `distutils` (or `setuptools`) to handle the
compilation step (see [cython documentation](http://docs.cython.org/)).

Apart from the boiler-plate import, I added type definitions to 
the function arguments. The cython types have the same name as 
numpy types with and additional `_t`. For the first argument
which is an array we use the syntax `np.float64_t[:] times`
that defines a cython [Memoryview](http://docs.cython.org/src/userguide/memoryviews.html)
(see also [Memoryview Benchmarks](https://jakevdp.github.io/blog/2012/08/08/memoryview-benchmarks/)).

Let's run the cython function:

In [14]:
bursts_cy = burst_search_cy(timestamps, 10, 100e-6)

In [15]:
assert bursts_cy == bursts_py

In [16]:
%timeit burst_search_cy(timestamps, 10, 100e-6)

100 loops, best of 3: 8.19 ms per loop


Note that the cython version executes in just ~10ms, a whooping 
100x speed increase compared to the pure python version. And all 
we have done is adding a few variable declarations!

Another optimization tool emegerged in the last couple of years is 
Numba. Numba is a Just-in-Time (JIT) compiler which analyze the code 
during the execution and translates it to machine code on-fly. Under 
the hood it uses the [LLVM compiler](https://en.wikipedia.org/wiki/LLVM) 
to translate python function to machine code.

In principle, numba can perform more advanced optimizations than 
cython and there are, in some cases, reports of 2x speed improvements 
vs cython.

Numba is even easier than cython to use, requiring only a single 
additional line, which is the decorator `@numba.jit`. Our example 
in numba becomes:

In [17]:
import numba

In [18]:
@numba.jit
def burst_search_numba(times, m, T):
    in_burst = False
    bursts = []
    start = 0.
    for i in range(times.size - m - 1):
        if times[i + m - 1] - times[i] <= T:
            if not in_burst:
                in_burst = True
                start = times[i]
        elif in_burst:
            in_burst = False
            bursts.append((start, times[i+m-1]))
    return bursts

Preatty neat.

The current drawback of using numba is that the automatic type 
inference does not work 100% of the times yet (remember that Numba 
is a relatively new project but evolving at an impressive pace). 
For example, in the previous function I had to add the line `start = 0.` 
to help numba infer the type of the `start` variable (an error is 
raised otherwise).

Also, in this particular case, numba does not show any speed 
improvement compared to the pure python version:

In [19]:
bursts_numba = burst_search_numba(timestamps, 10, 100e-6)

In [20]:
%timeit burst_search_numba(timestamps, 10, 100e-6)

1 loops, best of 3: 973 ms per loop


This "slow" execution happens because current numba (version 0.20) 
is not able to optimize this function to machine code and 
falls back to a pure python implementation.

I'm not sure what prevents numba to optimize this function 
(hey numba experts, please leave a comment!), but I would not 
be surprised if this corner cases will be handled pretty soon.

# Implementation 3: vectorized numpy

It may be not easily to spot at first glance, but we can also
get a some speed-up using pure python + numpy. We will not 
completely remove the for-loop but we will move some heavy 
computation out of it.

As shown before, the bottleneck is the operation 
`t[i+m] - t[i] <= T`. In particular the array element access 
is quite slow.

At cost of higher memory usage we can perform the difference 
outside the loop:

In [21]:
def burst_search_numpy1(times, m, T):
    in_burst = False
    bursts = []
    delta_t = (times[m-1:] - times[:times.size-m+1])
    for i in range(times.size-m+1):
        if delta_t[i] <= T:
            if not in_burst:
                in_burst = True
                start = times[i]
        elif in_burst:
            in_burst = False
            bursts.append((start, times[i+m-1]))
    return bursts

In [22]:
bursts_numpy1 = burst_search_numpy1(timestamps, 10, 100e-6)
assert bursts_numpy1 == bursts_py
%timeit burst_search_numpy1(timestamps, 10, 100e-6)

1 loops, best of 3: 416 ms per loop


we got a 2x speed improvement, not bad. 

Thinking further on this line, we can also move the comparison 
outside the loop:

In [23]:
def burst_search_numpy2(times, m, T):
    in_burst = False
    bursts = []
    above_min_rate = (times[m-1:] - times[:times.size-m+1]) <= T
    for i in range(times.size-m+1):
        test = above_min_rate[i]
        if test:
            if not in_burst:
                in_burst = True
                start = times[i]
        elif in_burst:
            in_burst = False
            bursts.append((start, times[i+m-1]))
    return bursts

In [24]:
bursts_numpy2 = burst_search_numpy2(timestamps, 10, 100e-6)
assert bursts_numpy2 == bursts_py
%timeit burst_search_numpy2(timestamps, 10, 100e-6)

1 loops, best of 3: 281 ms per loop


The line profiler shows that the simple item access (line 6) takes 30% of the time.
This is not surprising because numpy has some overhead when accessing a single
element because of all the fancy indexing it supports. Since here we are simply
iterating over all the elements we can use the iterator to get the array elements.


In [25]:
def burst_search_numpy3(times, m, T):
    in_burst = False
    bursts = []
    
    max_index = times.size-m+1
    above_min_rate = ((times[m-1:] - times[:max_index]) <= T)[:max_index]
    
    for i, above_min_rate_ in enumerate(above_min_rate):
        if above_min_rate_:
            if not in_burst:
                in_burst = True
                start = times[i]
        elif in_burst:
            in_burst = False
            bursts.append((start, times[i+m-1]))
    return bursts

In [26]:
bursts_numpy3 = burst_search_numpy3(timestamps, 10, 100e-6)
assert bursts_numpy3 == bursts_py
%timeit burst_search_numpy3(timestamps, 10, 100e-6)

1 loops, best of 3: 223 ms per loop


Finally, we can optimize line 13, the else branch that is executed at almost every cycle.
To avoid this test we can slightly unwrap the loop:

- first we `continue` if not above the rate.
- when "not continuing", we are inside a burst and we do a mini internal loop until the burst is over
- the main loop resumes from a position updated by the inner loop

The trick is using the same iterator in the two nested loops:

In [27]:
def burst_search_numpy4(times, m, T):
    bursts = []
    
    max_index = times.size-m+1
    above_min_rate = ((times[m-1:] - times[:max_index]) <= T)[:max_index]

    it = enumerate(above_min_rate)
    for i, above_min_rate_ in it:
        if not above_min_rate_: continue           
        
        start = times[i]
        for i, above_min_rate_ in it:
            if not above_min_rate_: break
        
        bursts.append((start, times[i+m-1]))
    return bursts

In [28]:
bursts_numpy4 = burst_search_numpy4(timestamps, 10, 100e-6)
assert bursts_numpy4 == bursts_py
%timeit burst_search_numpy4(timestamps, 10, 100e-6)

1 loops, best of 3: 205 ms per loop


This last two optimizations cut another 25% to the execution time.
More importantly, the code is easier to read, mainly because
we eliminated the state variable `in_burst`.

# Conclusion

We started with a direct pure-python algorithm implementation and we found
how to optimize it in cython obtaining a 100x (100 folds) speed-up.

The burst search algorithm is not easy to vectorize, but 
a careful analysis of the bottlenecks allowed 
to obtain a 5x speed-up using only python + numpy.

In most of cases, the speed of a python + numpy implementation 
is more than enough. For rare performance-critical functions, 
C-level performances can be achieved by using cython or numba.

But optimization must always be performed as last step, and only
focusing on the bottlenecks highlighted by the profiler.
When in doubt, prefer clarity over speed.

# See also

- [FRETBursts](http://tritemio.github.io/FRETBursts/) A burst analysis 
  software implementing the techniques illustrated in this post 
  in a more complete real-world scenario.